In [44]:
import pandas as pd
import datetime as dt
import numpy as np

# Начато 21.01.2022 в 19:17
ROW_DATA_PATH = "RawData/"
ROW_FILENAME = "2022-01-21-(012021-122021).xlsx"
# ROW_FILENAME = "Help.xlsx"
ROW_DATA_COLUMNS = {
                    "Дата" : "FDate",
                    "Функциональное направление" : "FNRow",
                    "МВЗ" : "MVZ",
                    "Направление" : "DivisionRow",
                    "Подразделение" : "SubDivision",
                    "Пользователь" : "User",
                    "Северный работник" : "Northern",
                    "Проект" : "Project",
                    "Статус проекта" : "ProjectState",
                    "Менеджер проекта" : "ProjectManager",
                    "Вид проекта" : "ProjectType",
                    "Кол-во штатных единиц" : "KoBo",
                    "План, FTE" : "PlanFTE",
                    "Договор" : "Contract",
                    "Фактические трудозатраты (час.) (Сумма)" : "FactHour",
                    "Unnamed: 15" : "Unnamed15",
                    "Unnamed: 16" : "Unnamed16"
                    }

RESULT_DATA_COLUMNS = [
                        "FDate",
                        "Month",
                        "FN",
                        "Division",
                        "User",
                        "Project",
                        "ProjectType",
                        "ProjectSubType",
                        "ProjectManager", 
                        "PlanFTE",
                        "FactFTE",
                        "Pdr_Proj_User_Month",
                        "Pdr_User_Proj_Month",
                        "Pdr_User_Month",
                        "Pdr_Proj_Month",
                        "ProjMang_Proj_Month",
                        "ProjMang_Proj_User_Month"
                      ]
ROW_DATA_DROP_COLUMNS = ["MVZ", "KoBo", "Contract", "Unnamed15", "Unnamed16"]
DONT_REPLACE_ENTER = ["Month"]

BOOLEAN_VALUES_SUBST = {"ЛОЖЬ": 0, "ИСТИНА": 1}

PARAMETERS_SECTION_NAME = "Parameters"
ROW_DATA_SECTION_NAME = "RowDataPath"
ROUND_FTE_SECTION_NAME = "RoundFTE"

MONTH_WORKING_HOURS_TABLE = "WHours.xlsx"
DIVISIONS_NAMES_TABLE = "ShortDivisionNames.xlsx"
FNS_NAMES_TABLE = "ShortFNNames.xlsx"
PROJECTS_SUB_TYPES_TABLE = "ProjectsSubTypes.xlsx"

ROUND_FTE_VALUE = 0
FACT_IS_PLAN_MARKER = "(факт=плану)"
OTHER_PROJECT_SUB_TYPE = "_Прочее"

p_delete_vacation = True

In [45]:
def get_parameter_value(paramname):
    # Читаем настройки
    settings_df = pd.read_excel("Settings.xlsx", engine='openpyxl')
    settings_df.dropna(how='all', inplace=True)
    ret_value = settings_df[settings_df["ParameterName"]==paramname]["ParameterValue"].to_list()[0]
    if type(ret_value) == str:
        ret_value = ret_value.replace("\\", "/")
        if ret_value[-1] == "/":
            ret_value = ret_value[:-1:]
    
    return (ret_value)

In [46]:
def load_row_data():
    # Загружаем сырые данные
    df = pd.read_excel(get_parameter_value(ROW_DATA_SECTION_NAME) + "/" + ROW_FILENAME, engine='openpyxl')
    df.dropna(how='all', inplace=True)
    df.rename(columns=ROW_DATA_COLUMNS, inplace=True)
    exist_drop_columns_list = list(set(ROW_DATA_DROP_COLUMNS) & set(df.dtypes.keys()))
    df.drop(columns=exist_drop_columns_list, inplace=True)

    return (df)

In [47]:
def load_parameter_table(tablename):
    # Загружаем соответствующу таблицу с параметрами
    parameter_df = pd.read_excel(get_parameter_value(PARAMETERS_SECTION_NAME) + "/" + tablename, engine='openpyxl')
    parameter_df.dropna(how='all', inplace=True)
    
    return(parameter_df)

In [48]:
def udata_2_date(data):
    if (data != data):
        ret_date = data
    elif type(data) == str:
        ret_date = dt.datetime.strptime("01."+data, '%d.%m.%Y')
    elif type(data) == float:
        ret_date = dt.datetime.strptime("01."+str(data), '%d.%m.%Y')
    else:
        ret_date = data
    
    return(ret_date)

In [49]:
def calc_fact_fte(FactHour, Northern, CHour, NHour, Project, PlanFTE):
    if Project.find(FACT_IS_PLAN_MARKER) >= 0:
        fact_fte = PlanFTE
    else:
        month_hours = NHour if Northern else CHour
        fact_fte = round(FactHour / month_hours, ROUND_FTE_VALUE)
    return(fact_fte)

In [50]:
def add_combine_columns(df):
# "Month",
# "FN",
# "Division",
# "User",
# "Project",
# "ProjectType",
# "ProjectSubType",
# "PlanFTE",
# "FactFTE"
# 
# Подразделение + Проект + ФИО + Месяц
# Подразделение + ФИО + Проект + Месяц
# Подразделение + ФИО + Месяц
# Подразделение + Проект + Месяц
# ПМ + Проект + Месяц
# ПМ + Проект + ФИО + Месяц
    df["ShortProject"] = df["Project"].str[:7]
    df["Pdr_Proj_User_Month"] = df["Division"] +"#"+ df["ShortProject"] +"#"+ df["User"] +"#"+ df["Month"]
    df["Pdr_User_Proj_Month"] = df["Division"] +"#"+ df["User"] +"#"+ df["ShortProject"] +"#"+ df["Month"]
    df["Pdr_User_Month"] = df["Division"] +"#"+ df["User"] +"#"+ df["Month"]
    df["Pdr_Proj_Month"] = df["Division"] +"#"+ df["ShortProject"] +"#"+ df["Month"]
    df["ProjMang_Proj_Month"] = df["ProjectManager"] +"#"+ df["ShortProject"] +"#"+ df["Month"]
    df["ProjMang_Proj_User_Month"] = df["ProjectManager"] +"#"+ df["ShortProject"] +"#"+ df["User"] +"#"+ df["Month"]
    
    

In [51]:
def prepare_data(data_df):
    for column_name in set(data_df.dtypes.keys()) - set(DONT_REPLACE_ENTER):
        if data_df.dtypes[column_name] == type(str):
            data_df[column_name] = data_df[column_name].str.replace("\n", "")
    
    data_df["FDate"] = data_df["FDate"].apply(lambda param: udata_2_date(param))
    data_df['Northern'].replace(BOOLEAN_VALUES_SUBST, inplace=True)
    data_df = data_df.merge(month_hours_df, left_on="FDate", right_on="FirstDate", how="inner")
    
    data_df["FactFTE"] = \
        data_df[["FactHour", "Northern", "CHour", "NHour", "Project", "PlanFTE"]].apply( \
            lambda param: calc_fact_fte(*param), axis=1
        )

    data_df = data_df.merge(divisions_names_df, left_on="DivisionRow", right_on="FullDivisionName", how="left")
    data_df["Division"] = data_df[["ShortDivisionName", "DivisionRow"]].apply(lambda param: param[1] if pd.isna(param[0]) else param[0], axis=1)

    data_df = data_df.merge(fns_names_df, left_on="FNRow", right_on="FullFNName", how="left")
    data_df["FN"] = data_df[["ShortFNName", "FNRow"]].apply(lambda param: param[1] if pd.isna(param[0]) else param[0], axis=1)
    
    data_df["ProjectType"] = \
        data_df[["Project", "ProjectType"]].apply( \
            lambda param: "S" if param[0].find(FACT_IS_PLAN_MARKER)>=0 else param[1], axis=1)

    data_df = data_df.merge(projects_sub_types_df, left_on="Project", right_on="ProjectName", how="left")
    data_df["ProjectSubType"] = \
        data_df[["ProjectType", "ProjectSubTypePart"]].apply( \
            lambda param: param[0]+OTHER_PROJECT_SUB_TYPE if pd.isna(param[1]) else param[1], axis=1 \
                                                        )
    
    if p_delete_vacation:
        data_df["User"] = \
            data_df["User"].apply(lambda param: "вакансия" if param.replace(" ","").lower()[:8]=="вакансия" else param)
        
        data_df = data_df[data_df["User"]!="вакансия"]
   
    add_combine_columns(data_df)
    
    return(data_df[RESULT_DATA_COLUMNS])
    return(data_df)
    


In [52]:
row_df = load_row_data()
month_hours_df = load_parameter_table(MONTH_WORKING_HOURS_TABLE)
divisions_names_df = load_parameter_table(DIVISIONS_NAMES_TABLE)
fns_names_df = load_parameter_table(FNS_NAMES_TABLE)
projects_sub_types_df = load_parameter_table(PROJECTS_SUB_TYPES_TABLE)

ROUND_FTE_VALUE = get_parameter_value(ROUND_FTE_SECTION_NAME)

In [53]:
report_df = prepare_data(row_df.copy())


In [54]:
report_df.head()

,FDate,Month,FN,Division,User,Project,ProjectType,ProjectSubType,ProjectManager,PlanFTE,FactFTE,Pdr_Proj_User_Month,Pdr_User_Proj_Month,Pdr_User_Month,Pdr_Proj_Month,ProjMang_Proj_Month,ProjMang_Proj_User_Month
0,2021-01-01,янв,HCM,НПИС,Галустьян Инна Суреновна,Т0001-АКП,Т,Т_Прочее,Пуромов Евгений Владимирович,0.10,0.000,НПИС#Т0001-А#Галустьян Инна Суреновна#янв,НПИС#Галустьян Инна Суреновна#Т0001-А#янв,НПИС#Галустьян Инна Суреновна#янв,НПИС#Т0001-А#янв,Пуромов Евгений Владимирович#Т0001-А#янв,Пуромов Евгений Владимирович#Т0001-А#Галустьян...
1,2021-01-01,янв,HCM,НПИС,Кравченко Михаил Александрович,Т0001-АКП,Т,Т_Прочее,Пуромов Евгений Владимирович,0.10,0.000,НПИС#Т0001-А#Кравченко Михаил Александрович#янв,НПИС#Кравченко Михаил Александрович#Т0001-А#янв,НПИС#Кравченко Михаил Александрович#янв,НПИС#Т0001-А#янв,Пуромов Евгений Владимирович#Т0001-А#янв,Пуромов Евгений Владимирович#Т0001-А#Кравченко...
2,2021-01-01,янв,HCM,ДИСУ,Пуромов Евгений Владимирович,Т0001-АКП,Т,Т_Прочее,Пуромов Евгений Владимирович,0.03,0.030,ДИСУ#Т0001-А#Пуромов Евгений Владимирович#янв,ДИСУ#Пуромов Евгений Владимирович#Т0001-А#янв,ДИСУ#Пуромов Евгений Владимирович#янв,ДИСУ#Т0001-А#янв,Пуромов Евгений Владимирович#Т0001-А#янв,Пуромов Евгений Владимирович#Т0001-А#Пуромов Е...
3,2021-01-01,янв,ГиД,ДКиПС,(Уволен) Папулин Евгений Сергеевич,Т0003-База геоданных ГИС,Т,Т_Прочее,Заславская Ярослава Владимировна,0.14,0.000,ДКиПС#Т0003-Б#(Уволен) Папулин Евгений Сергеев...,ДКиПС#(Уволен) Папулин Евгений Сергеевич#Т0003...,ДКиПС#(Уволен) Папулин Евгений Сергеевич#янв,ДКиПС#Т0003-Б#янв,Заславская Ярослава Владимировна#Т0003-Б#янв,Заславская Ярослава Владимировна#Т0003-Б#(Увол...
4,2021-01-01,янв,HCM,ДИСУ,Бударин Роман Владимирович,Т0004-БОСС-Кадровик АГД ДАЙМОНДС,Т,Т_Прочее,Пуромов Евгений Владимирович,0.03,0.025,ДИСУ#Т0004-Б#Бударин Роман Владимирович#янв,ДИСУ#Бударин Роман Владимирович#Т0004-Б#янв,ДИСУ#Бударин Роман Владимирович#янв,ДИСУ#Т0004-Б#янв,Пуромов Евгений Владимирович#Т0004-Б#янв,Пуромов Евгений Владимирович#Т0004-Б#Бударин Р...


In [55]:
# Import `dataframe_to_rows`
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
from openpyxl import Workbook
import sys

# Load in the workbook
wb = load_workbook('example.xlsx')
# Get sheet names
print(wb.sheetnames)

help_df = report_df
for row in dataframe_to_rows(help_df, index=False, header=False):
    wb['ИсходныеДанные'].append(row)

try:
    wb.save('example2.xlsx')
except:
    print("Файл не доступен. Видимо он открыт. Пожалуйста закройте его в Excel.")
    e = sys.exc_info()[1]
    print(e.args[0])


['Sheet3', 'ИсходныеДанные']


In [1]:
"data.txt".endswith(".txt")

True